# IEEE-CIS Fraud Detection

## Feature Engineering

In [2]:
# Import necessary modules

import pandas as pd
import numpy as np

In [75]:
train = pd.read_pickle('dataset/train_mem.pkl')
train.set_index('TransactionID', drop=True, inplace=True)

test = pd.read_pickle('dataset/test_mem.pkl')
test.set_index('TransactionID', drop=True, inplace=True)

In [76]:
def feature_engineering(train, test):
    
    ## PREPROCESSING
    # Preprocessing for unique values in the test set (e-mail columns)
    P_email_list = train.P_emaildomain.unique().tolist()
    test.P_emaildomain.loc[~test.P_emaildomain.isin(P_email_list)] = np.nan
    
    R_email_list = train.R_emaildomain.unique().tolist()
    test.R_emaildomain.loc[~test.R_emaildomain.isin(R_email_list)] = np.nan
    
    # Concatenate train and test data
    data = pd.concat((train.drop('isFraud', axis=1), test)).copy()

    ## FEATURE SELECTION
    # List of columns/features to be removed from the adtaaset
    remove_list = (#['TransactionAmt'] # new (different) variable created
                   ['C2', 'C4', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14'] # correlation
                  +['D2', 'D6', 'D7', 'D12'] # correlation
                  +['V5', 'V9', 'V11', 'V13', 'V16', 'V17', 'V18', 'V20', 'V21', 'V22', 'V24', 
                    'V26', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V36', 'V38', 'V39', 
                    'V40', 'V41', 'V42', 'V43', 'V45', 'V48', 'V49', 'V50', 'V51', 'V52', 'V54', 
                    'V57', 'V58', 'V59', 'V60', 'V62', 'V63', 'V64', 'V65', 'V67', 'V68', 'V69', 
                    'V70', 'V71', 'V72', 'V73', 'V74', 'V76', 'V79', 'V80', 'V81', 'V83', 'V84', 
                    'V85', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V96', 'V97', 
                    'V100', 'V101', 'V102', 'V103', 'V105', 'V106', 'V110', 'V113', 'V116', 'V119', 
                    'V125', 'V126', 'V127', 'V128', 'V132', 'V133', 'V134', 'V137', 'V140', 'V142', 
                    'V143', 'V145', 'V147', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 
                    'V156', 'V157', 'V158', 'V159', 'V160', 'V162', 'V163', 'V164', 'V165', 'V167', 
                    'V168', 'V177', 'V178', 'V179', 'V180', 'V181', 'V182', 'V183', 'V185', 'V186', 
                    'V189', 'V190', 'V191', 'V192', 'V193', 'V195', 'V196', 'V197', 'V198', 'V199', 
                    'V200', 'V201', 'V202', 'V203', 'V204', 'V206', 'V207', 'V211', 'V212', 'V213', 
                    'V216', 'V217', 'V218', 'V219', 'V222', 'V225', 'V228', 'V229', 'V230', 'V231', 
                    'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V239', 'V242', 'V243', 'V244', 
                    'V245', 'V246', 'V247', 'V248', 'V249', 'V251', 'V252', 'V253', 'V254', 'V255', 
                    'V256', 'V257', 'V258', 'V259', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 
                    'V267', 'V268', 'V269', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 
                    'V278', 'V279', 'V280', 'V285', 'V287', 'V289', 'V292', 'V293', 'V294', 'V295', 
                    'V296', 'V297', 'V298', 'V299', 'V301', 'V302', 'V303', 'V304', 'V306', 'V307', 
                    'V308', 'V309', 'V310', 'V311', 'V312', 'V315', 'V316', 'V317', 'V318', 'V319', 
                    'V320', 'V321', 'V322', 'V323', 'V324', 'V325', 'V326', 'V327', 'V328', 'V329', 
                    'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339'] # correlation
                  ## FOR NOW
                  +['DeviceType', 'DeviceInfo'] # Needs feature engineering and missing val analysis
                  +['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 
                     'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 
                     'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 
                     'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 
                     'id_37', 'id_38'] # Needs feature engineering and missing val analysis
                  ) 
    # Remove unnecessary columns
    data.drop(remove_list, axis=1, inplace=True)
    
    
    ## TRANSACTION 
    # Create a new column with the log of TransactionAmt
    data['TransactionAmt_log'] = np.log(data.TransactionAmt)
    
    # Datetime
    import datetime
    START_DATE = '2019-03-01'
    startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
    data['TransactionDT_DT'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
    
    # Hour column
    hours = data['TransactionDT']/3600
    data['Transaction_hours'] = np.floor(hours) % 24

    ## E-MAIL 
    # Fill NaNs with "missing"
    data.P_emaildomain.fillna('missing', inplace=True)
    data.P_emaildomain.fillna('missing', inplace=True)

    # Create dict for each columns (using train data)
    P_email_dict = train.groupby('P_emaildomain').isFraud.mean().to_dict()
    R_email_dict = train.groupby('R_emaildomain').isFraud.mean().to_dict()

    # Create new numeric columns
    data['P_emaildomain_val'] = data.P_emaildomain.map(P_email_dict)
    data['R_emaildomain_val'] = data.R_emaildomain.map(R_email_dict)
    
 
    ## PREPARE DATA FOR MODEL
    # List of columns to be dropped from the dataset for modeling
    drop_list = (['TransactionDT'] # new (same) variable created
                +['TransactionDT_DT'] # datetime 
                +['P_emaildomain', 'R_emaildomain'] # new (different) variable created
                )
    
    # Create a list of columns with more than 20% missing 
    remove_missing_cols = data.isna().mean()[data.isna().mean()>0.5].index.to_list()
    
    # Drop unnecessary columns 
    data.drop(drop_list+remove_missing_cols, axis=1, inplace=True)
    
    # Create dummy variables
    data = pd.get_dummies(data)
    
    # Split train and test data
    train_data = data[:train.shape[0]]
    test_data = data[train.shape[0]:]
    
    return train_data.join(train.isFraud), test_data

In [77]:
train, test = feature_engineering(train, test)

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [78]:
train.shape, test.shape

((590540, 110), (506691, 109))

### Save Train and Test Data

In [79]:
pd.to_pickle(train, 'dataset/train_engineered_new.pkl')
pd.to_pickle(test, 'dataset/test_engineered_new.pkl')

### Feature Selection

In [5]:
# List of columns/features to be removed from the model
remove_list = (#['TransactionAmt'] # new (different) variable created
               ['TransactionDT_DT'] # datetime 
              +['C2', 'C4', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14'] # correlation
              +['D2', 'D6', 'D7', 'D12'] # correlation
              +['P_emaildomain', 'R_emaildomain'] # new (different) variable created
              +['V5', 'V9', 'V11', 'V13', 'V16', 'V17', 'V18', 'V20', 'V21', 'V22', 'V24', 
                'V26', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V36', 'V38', 'V39', 
                'V40', 'V41', 'V42', 'V43', 'V45', 'V48', 'V49', 'V50', 'V51', 'V52', 'V54', 
                'V57', 'V58', 'V59', 'V60', 'V62', 'V63', 'V64', 'V65', 'V67', 'V68', 'V69', 
                'V70', 'V71', 'V72', 'V73', 'V74', 'V76', 'V79', 'V80', 'V81', 'V83', 'V84', 
                'V85', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V96', 'V97', 
                'V100', 'V101', 'V102', 'V103', 'V105', 'V106', 'V110', 'V113', 'V116', 'V119', 
                'V125', 'V126', 'V127', 'V128', 'V132', 'V133', 'V134', 'V137', 'V140', 'V142', 
                'V143', 'V145', 'V147', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 
                'V156', 'V157', 'V158', 'V159', 'V160', 'V162', 'V163', 'V164', 'V165', 'V167', 
                'V168', 'V177', 'V178', 'V179', 'V180', 'V181', 'V182', 'V183', 'V185', 'V186', 
                'V189', 'V190', 'V191', 'V192', 'V193', 'V195', 'V196', 'V197', 'V198', 'V199', 
                'V200', 'V201', 'V202', 'V203', 'V204', 'V206', 'V207', 'V211', 'V212', 'V213', 
                'V216', 'V217', 'V218', 'V219', 'V222', 'V225', 'V228', 'V229', 'V230', 'V231', 
                'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V239', 'V242', 'V243', 'V244', 
                'V245', 'V246', 'V247', 'V248', 'V249', 'V251', 'V252', 'V253', 'V254', 'V255', 
                'V256', 'V257', 'V258', 'V259', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 
                'V267', 'V268', 'V269', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 
                'V278', 'V279', 'V280', 'V285', 'V287', 'V289', 'V292', 'V293', 'V294', 'V295', 
                'V296', 'V297', 'V298', 'V299', 'V301', 'V302', 'V303', 'V304', 'V306', 'V307', 
                'V308', 'V309', 'V310', 'V311', 'V312', 'V315', 'V316', 'V317', 'V318', 'V319', 
                'V320', 'V321', 'V322', 'V323', 'V324', 'V325', 'V326', 'V327', 'V328', 'V329', 
                'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339'] # correlation
              ## FOR NOW
              + ['DeviceType', 'DeviceInfo'] # Needs feature engineering and missing val analysis
              + ['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 
                 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 
                 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 
                 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 
                 'id_37', 'id_38'] # Needs feature engineering and missing val analysis
              ) 

# List of columns to be removed from the dataset for good
drop_list = (['TransactionDT'] # new (same) variable created
            )

### Transaction

In [6]:
# Create a new column with the log of TransactionAmt
train['TransactionAmt_log'] = np.log(train.TransactionAmt)

In [7]:
import datetime

START_DATE = '2019-03-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')

train['TransactionDT_DT'] = train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))

In [8]:
def make_hour_feature(df, tname='TransactionDT'):
    """
    Creates an hour of the day feature, encoded as 0-23. 
    
    Parameters:
    -----------
    df : pd.DataFrame
        df to manipulate.
    tname : str
        Name of the time column in df.
    """
    hours = df[tname] / (3600)        
    encoded_hours = np.floor(hours) % 24
    return encoded_hours

train['Transaction_hours'] = make_hour_feature(train)

### E-mail

In [9]:
# Fill NaNs with "missing"
train.P_emaildomain.fillna('missing', inplace=True)
train.R_emaildomain = train.P_emaildomain.fillna('missing')

# Create dict for each columns (-> use these dicts in for converting TEST DATA)
P_email_dict = train.groupby('P_emaildomain').isFraud.mean().to_dict()
R_email_dict = train.groupby('R_emaildomain').isFraud.mean().to_dict()

# Create new numeric columns
train['P_emaildomain_val'] = train.P_emaildomain.map(P_email_dict)
train['R_emaildomain_val'] = train.R_emaildomain.map(R_email_dict)

## Encoding

In [11]:
from sklearn.preprocessing import OneHotEncoder

# Create the encoder.
encoder = OneHotEncoder(handle_unknown="ignore")
encoder.fit(train)

# Apply the encoder.
train_encoded = encoder.transform(X_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## Imputation

In [ ]:
from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# imputer.fit_transform(df)

## Save Data

In [10]:
train.shape

(590540, 439)

In [11]:
len(remove_list + drop_list)

295

In [12]:
save_train = train.drop(remove_list + drop_list, axis=1)

In [13]:
save_train.shape

(590540, 144)

In [14]:
pd.to_pickle(save_train, 'dataset/train_engineered.pkl')